In [1]:
from pyrosm import OSM, get_data
import osmnx
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import networkx as nx

/opt/python/envs/default/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
reston = OSM("planet_-77.703,38.783_-77.018,39.127.osm.pbf")

In [3]:
nodes, edges = reston.get_network(network_type="driving", nodes=True)

In [4]:
speed_limits = {
    "residential": 30,
    "motorway": 55,
    "road": 40
}

In [5]:
def determine_max_speed(row):
    if row.maxspeed:
        if ";" in row.maxspeed:
            spd = row.maxspeed.split(";")[0]
        else:
            spd = row.maxspeed

        return float(spd.replace(" mph", ""))
    elif row.highway in list(speed_limits.keys()):
        return speed_limits[row.highway]
    else:
        return 35.0

In [6]:
edges["max_mph"] = edges.apply(determine_max_speed, axis=1)

In [7]:
edges.geometry = edges.geometry.to_crs("+proj=cea")
nodes.geometry = nodes.geometry.to_crs("+proj=cea")

In [8]:
edges = edges[["geometry", "u", "v", "length", "max_mph", "highway", "oneway"]].to_crs("+proj=cea")
nodes = nodes.to_crs("+proj=cea")

In [9]:
edges

,geometry,u,v,length,max_mph,highway,oneway
0,"LINESTRING (-8576972.435 3979717.996, -8576983...",1222438006,2838546923,11.660,35.0,service,None
1,"LINESTRING (-8576983.879 3979712.134, -8577003...",2838546923,1222438044,20.599,35.0,service,None
2,"LINESTRING (-8577003.193 3979701.163, -8577019...",1222438044,1222437980,14.672,35.0,service,None
3,"LINESTRING (-8577332.431 3979794.878, -8577319...",262247665,1797633369,9.727,35.0,service,None
4,"LINESTRING (-8577319.941 3979795.199, -8577253...",1797633369,1222437964,51.769,35.0,service,None
...,...,...,...,...,...,...,...
1151582,"LINESTRING (-8599639.277 3982379.376, -8599625...",9762896753,9762896750,10.364,35.0,service,None
1151583,"LINESTRING (-8600979.574 3982915.024, -8600968...",2404261592,9762908929,12.261,35.0,service,yes
1151584,"LINESTRING (-8600968.665 3982921.913, -8600960...",9762908929,9762908930,7.681,35.0,service,yes
1151585,"LINESTRING (-8600960.695 3982925.444, -8600929...",9762908930,9762908932,24.512,35.0,service,yes


In [10]:
edges["travel_time"] = (edges.length * 0.000621371)/(edges.max_mph / 60)

In [11]:
edges.geometry = edges.geometry.to_crs(4326)
nodes.geometry = nodes.geometry.to_crs(4326)

In [12]:
dgraph = reston.to_graph(nodes, edges, graph_type="networkx")

In [13]:
osmnx.project_graph(dgraph, to_crs=4326)

In [14]:
source_address = "1946 Isaac Newton Square W, Reston, Virginia"
sourcey, sourcex = osmnx.geocode(source_address)

In [15]:
monkeyhouse, distance_to_monkeyhouse = osmnx.distance.nearest_nodes(dgraph, X=sourcex, Y=sourcey, return_dist=True)
travel_times = nx.single_source_dijkstra_path_length(dgraph, source=monkeyhouse, weight="travel_time")

In [16]:
times = pd.DataFrame([list(travel_times.keys()), list(travel_times.values())]).T

In [17]:
times.columns = ["node_id", "travel_time"]

In [18]:
n = nodes.merge(times, left_on="id", right_on="node_id")

In [19]:
n

,lon,lat,tags,timestamp,version,changeset,id,geometry,node_id,travel_time
0,-77.048254,38.833886,None,0,0,0,1222438006,POINT (-77.04825 38.83389),1.222438e+09,29.756144
1,-77.048357,38.833818,None,0,0,0,2838546923,POINT (-77.04836 38.83382),2.838547e+09,29.769840
2,-77.048531,38.833691,None,0,0,0,1222438044,POINT (-77.04853 38.83369),1.222438e+09,29.793501
3,-77.048677,38.833624,None,0,0,0,1222437980,POINT (-77.04868 38.83362),1.222438e+09,29.796457
4,-77.051488,38.834774,"{'highway': 'traffic_signals', 'source': 'surv...",0,0,0,262247665,POINT (-77.05149 38.83477),2.622477e+08,29.648222
...,...,...,...,...,...,...,...,...,...,...
1060413,-77.251900,38.864629,None,0,0,0,9762896749,POINT (-77.25190 38.86463),9.762897e+09,15.966225
1060414,-77.251874,38.864628,{'barrier': 'gate'},0,0,0,9762896753,POINT (-77.25187 38.86463),9.762897e+09,15.963094
1060415,-77.263816,38.870897,None,0,0,0,9762908929,POINT (-77.26382 38.87090),9.762909e+09,14.474132
1060416,-77.263745,38.870937,None,0,0,0,9762908930,POINT (-77.26374 38.87094),9.762909e+09,14.483418


In [20]:
arterial_roads = edges.loc[edges.highway.isin(["motorway", "trunk", "primary", "primary_link"])]

In [ ]:
fig = plt.figure(facecolor="w", figsize=(10, 5), dpi=600)
ax = fig.add_subplot(111, axisbelow=True)

arterial_roads.plot(ax=ax, lw=0.5, color='gray', zorder=3)
ax.plot(sourcex, sourcey, ".k")
ax = n.plot('travel_time', ax=ax, cmap="inferno_r", markersize=2, legend=True, alpha = 0.06)
ax.set_xlim(-77.65, -77.05)
ax.set_ylim(38.8, 39.1)

plt.savefig("reston_virus.pdf")

Error: Error: Error processing request